# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: This is a binary classification problem as we need to classify students into two categories: the ones that need intervention and the ones that don't. **


## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [53]:
# TODO: Calculate number of students
n_students = len(student_data.index)
# print student_data.shape

# TODO: Calculate number of features, the last column "passed" is the label
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students
n_passed = student_data[student_data["passed"] == "yes"].count()["passed"]

# TODO: Calculate failing students
n_failed = student_data[student_data["passed"] == "no"].count()["passed"]

# TODO: Calculate graduation rate
from __future__ import division
grad_rate = (n_passed/n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [20]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print "\nLabels values:"
print y_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [26]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [54]:
# TODO: Import any additional functionality you may need here

# TODO: Set the number of training points
from sklearn.cross_validation import train_test_split
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, stratify=y_all, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:

A. Gaussian Naive Bayes
* Real-world application: Disease detection can use the Naive Bayes Classifier to classify whether or not a patient has certain disease based on past data
* Strengths: Simple, fast to train, fast to run on test data, does not require a lot of data to calculate parameters because the decoupling of features
* When to use: when the noise model complies with Gaussian distribution, and features are independent, hence the naive part. 
* Weaknesses: It is known to be a bad predictor for regression problem
* When not to use: regression problem and when features are clearly not independent
* Why the model for this problem given data: the features in the data are fairly independent and it is a classification problem.

B. AdaBoost 
* Real-world application: Face and shape recognition
* Strengths: Powerful ensemble method that fits a sequence of weak learners. It does not over-fit. 
* When to use: when you don't care training time as much and need a strong estimator, it can be used for both classification and regression
* Weaknesses: Can take a long time to train when you have a lot of iterations, sensitive of outliers
* When not to use: When your data is really noisy and you want a estimator that has fast training time 
* Why the model for this problem given data: We want a strong estimator that is resistent to overfitting

C. Support Vector Machine
* Real-world application: Natural language processing
* Strengths: effective in high dimension space, memory efficient with support vectors, versatile with different kernel functions
* When to use: when you want a fast training algorithm that applies to high dimension space(with many features), and when you have a dataset with realtively low percentage of noise
* Weaknesses: when features are more than samples, tend to give poor performance, sensitive to outliers
* When not to use: when you want a really strong estimator and when you have more features than samples in the dataset
* Why the model for this problem given data: we have 48 features with 300 samples for training, so it is sufficient for SVM to perform well in the high dimension space
**

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [55]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [71]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C

#Normalize the features
from sklearn.preprocessing import Normalizer
normer = Normalizer()
X_train = normer.fit_transform(X_train)
X_test = normer.transform(X_test)

# TODO: Initialize the three models
from sklearn.naive_bayes import GaussianNB
clf_A = GaussianNB()

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
#Default for AdaBoostClassifier is SAMME.R, 50 estimators and learning rate 1.0
#Default for the base estimator DecisionTreeClassifier is min_samples_split = 2, and gini as criterion
clf_B =  AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, random_state=42), random_state=42)

from sklearn.svm import SVC
clf_C = SVC(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]
#print X_train_100.shape

X_train_200 = X_train[99:299]
y_train_200 = y_train[99:299]
#print X_train_200.shape

X_train_300 = X_train
y_train_300 = y_train

#X_train, X_test, y_train, y_test

# TODO: Execute the 'train_predict' function for each classifier and each training set size

for clf in [clf_A, clf_B, clf_C]:

    # show the estimator type
    print "\n{}: \n".format(clf.__class__.__name__)

    # TODO: loop thru training sizes
    for n in [100, 200, 300]:

        # fit model using "n" training data points (i.e., 100, 200, or 300)
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)




    


GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0006 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.7939.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6870.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0006 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8045.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7176.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0007 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8145.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7442.

AdaBoostClassifier: 

Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.1193 seconds
Made predictions in 0.0041 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0041 seconds.
F1 score for test set: 0.6870.
Training a AdaBoostClassifier usin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0013             |        0.0005                |     0.8467             |    0.8029            |
| 200               |        0.0007          |       0.0003                 |      0.8188            |     0.7097            |
| 300               |        0.0008                 |         0.0003               |     0.8038             |   0.7634      |

** Classifer 2 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.1054                  |        0.0041                |      1.0000            |            0.7273     |
| 200               |       0.1063          |        0.0041                |    1.0000              |       0.7353          |
| 300               |         0.1128                |     0.0044                   |      0.9951            |    0.7442      |

** Classifer 3 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0011                |         0.0008               |      0.8777            |   0.7746              |
| 200               |         0.0027                |          0.0009              |      0.8889            |    0.7919             |
| 300               |          0.0056               |            0.0014            |        0.8761          |    0.7838             |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: Based on experiments, I found that the SVM performed the best among the three classifiers I have chosen, and is potentially the best model for student_intervention problem. In terms of the attributes of the data, it is easy to see that there are a lot of features to consider, and SVM is a good estimator on higher dimension space with various kernel tricks as long as there are sufficient data. In terms of limited resources and cost, SVM is very memory efficient as it only cares about a few support vectors, and it typically trains faster than adaboost. It is also important to notice that the features in the dataset are not independent as we have converted categorical variables into multiple columns, and results in poorer performance for GaussianNBClassifier. In contrast, SVM is a measure of similarities maximizing the margins amongst classes that do not care about the independence of features. In terms of performance, SVM has the highest test score with 300 traning samples.  **

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: SVM is a popular algorithm for classification problem. The SVM takes data about previous students (age, gender, family, etc), and uses them to create a function that draws a boundary between the students who passed and those who didn't. The boundary should be drawn so as to maximize the difference between students who passed and who didn't while ensuring that majority of samples are correctly classified. Often, though, it's not easy to draw a decision boundary in low dimensions, so the SVM separates the passing and failing students by transforming the input space into a higher dimension feature space. Using this function created with students we already know passed or not, the SVM can look at new students' data and predict whether the student needs intervention based on the student's various features(age, gender, family, etc). SVM is very efficient for classification problem since it only cares about the samples that are closer to the decision boundary.**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [82]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
#The gamma parameters can be seen as the inverse of the radius of influence of samples selected by the model as support vectors.
#The higher the gamma, the less influence the further points from decision boundary have
#C is the tradeoff between smoothness and correctness of classification for traning dataset
parameters = {'kernel': ['linear', 'rbf', 'sigmoid'], 'C':[1, 5, 10, 100], 'gamma': ['auto', 0, 0.1, 0.5, 1]}

#Normalize the features
from sklearn.preprocessing import Normalizer
normer = Normalizer()
X_train = normer.fit_transform(X_train)
X_test = normer.transform(X_test)

# TODO: Initialize the classifier
from sklearn.svm import SVC
clf = SVC(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
from sklearn.metrics import f1_score
f_one_scorer = make_scorer(f1_score, pos_label='yes', average='binary')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f_one_scorer, cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train_300, y_train_300)
from IPython.display import display
print "grid scores"
display(pd.DataFrame(grid_obj.grid_scores_))

# Get the estimator
clf = grid_obj.best_estimator_
print "best classifier parameters"
print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

grid scores


,parameters,mean_validation_score,cv_validation_scores
0,"{u'kernel': u'linear', u'C': 1, u'gamma': u'au...",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
1,"{u'kernel': u'rbf', u'C': 1, u'gamma': u'auto'}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
2,"{u'kernel': u'sigmoid', u'C': 1, u'gamma': u'a...",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
3,"{u'kernel': u'linear', u'C': 1, u'gamma': 0}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
4,"{u'kernel': u'rbf', u'C': 1, u'gamma': 0}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
5,"{u'kernel': u'sigmoid', u'C': 1, u'gamma': 0}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
6,"{u'kernel': u'linear', u'C': 1, u'gamma': 0.1}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
7,"{u'kernel': u'rbf', u'C': 1, u'gamma': 0.1}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
8,"{u'kernel': u'sigmoid', u'C': 1, u'gamma': 0.1}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"
9,"{u'kernel': u'linear', u'C': 1, u'gamma': 0.5}",0.802387,"[0.803921568627, 0.8, 0.8, 0.8, 0.808080808081]"


best classifier parameters
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0029 seconds.
Tuned model has a training F1 score of 0.8288.
Made predictions in 0.0009 seconds.
Tuned model has a testing F1 score of 0.7919.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: The final F1 score for training is 0.8288, and 0.7919 for testing. The final SVC model uses rbf kernel with a C value of 10, and gamma value of 0.0. The untuned model has F1 score for training is 0.8664, and 0.8052  for testing. Compared to the untuned model, which uses rbf kernel with a C value of 1, the tuned model after using GridSearch method actually has a lower F1 score for testing. This is interesting because the best estimator given by grid_search has a lower f1 score on the testing set than another estimator deemed as not as good as the best estimator from grid_search.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.